# 1. Setting

### 1.1 Import List

In [2]:
import torch
from torchvision import transforms, datasets, models
import torch.nn as nn
import torch.optim as optim

### 1.2 전이학습(VGG16) 모델 불러오기

전이 학습 모델은 다양하나 본 데이터셋의 Size(227x227) 구성과 가장 유사한 VGG16(224x224) 모델을 체택하였다.

In [3]:
# VGG 모델 불러오기
vgg = models.vgg16(pretrained=True)

# 모델의 마지막 층을 새로운 분류 층으로 대체한다.
# ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']

num_classes = 7
vgg.classifier[-1] = nn.Linear(in_features=4096, out_features=num_classes) #  vgg : 4096개의 입력 뉴런 보유

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 84.9MB/s]


In [4]:
# GPU 사용 권장.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg.to(device) # 모델을 device로 이동

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 2. Dataset Introduce

### 2.1 데이터 불러오기

In [5]:
# 데이터셋 압축 해제
import zipfile
import os

# Test Dataset
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Dataset/Art_Author_2/test.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

# Train Dataset
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Dataset/Art_Author_2/train.zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

### 2.2 데이터 분할

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# train 폴더 명 raw_train 변환
os.rename("/content/train", "/content/raw_train")
dataset_folder = './raw_train'

# 해당 경로 raw_train 훈련/검증 셋 분할 예정
train_folder = './train'
val_folder =  './valid'

In [7]:
# 폴더 명(클래스) list 생성
# 아래 폴더로 구분 ['horse', 'giraffe', 'person', 'house', 'guitar', 'dog', 'elephant']
class_folders = [folder for folder in os.listdir(dataset_folder) if os.path.isdir(os.path.join(dataset_folder, folder))]

# 각 클래스로 데이터 이동
for class_folder in class_folders:
    class_folder_path = os.path.join(dataset_folder, class_folder)

    # 클래스 폴더 안의 파일 목록 가져오기
    file_list = os.listdir(class_folder_path)

    # 훈련 / 검증 데이터 나누기
    train_files, val_files = train_test_split(file_list, test_size=0.2, random_state=2024)

    # 훈련 데이터를 train 폴더로 이동
    for file_name in train_files:
        src = os.path.join(class_folder_path, file_name)
        dst = os.path.join(train_folder, class_folder, file_name)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

    # 검증 데이터를 val 폴더로 이동
    for file_name in val_files:
        src = os.path.join(class_folder_path, file_name)
        dst = os.path.join(val_folder, class_folder, file_name)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

In [8]:
# 데이터셋 경로와 변환 정의

train_dir = './train'
val_dir = './valid'

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG는 224x224 크기의 이미지를 입력으로 사용
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # 관련 선행 사례 추천해준 값으로 설정
])

# 데이터셋 불러오기
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

# 데이터 로더 설정
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False) #  검증 데이터는 모델의 일반화 성능을 평가하기 위해 사용되므로 섞이지 않는 것이 일반적이다.

## 2. Model Setting

### 2.1 손실 함수 및 optimizer 설정

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.parameters(), lr=0.001, momentum=0.9)

# 에폭 수 지정
num_epochs = 20

# 모델 학습
for epoch in range(num_epochs):
    vgg.train() # VGG 신경망을 훈련 모드로 설정
    running_loss = 0.0 # 손실 초기화
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device) # GPU
        optimizer.zero_grad() # 옵티마이저 gradient 초기화
        outputs = vgg(inputs) # VGG 입력전달 및 출력값 반환
        loss = criterion(outputs, labels) # 출력과 실제 레이블 간의 손실을 계산
        loss.backward() # 역전파를 통해 그래디언트를 계산
        optimizer.step() # 옵티마이저를 사용하여 모델 파라미터를 업데이트
        running_loss += loss.item() # 현재 배치의 손실을 러닝 손실에 합산

    # 검증 세트를 사용하여 검증 정확도 계산
    vgg.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vgg(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Training Loss: {running_loss/len(train_loader):.4f}, '
          f'Validation Accuracy: {val_accuracy:.2f}%')

# early stop 같은 기능들을 넣는다면 110점짜리 완성
# 우선 딥러닝 모델을 구성 할 때에 suffle을 실행하나 다시 한 번 모델 구성 과정에서 suffle을 함으로 인해 매번 결과는 다소 상이하게 나올 수 있다.


Epoch [1/20], Training Loss: 0.9895, Validation Accuracy: 87.98%
Epoch [2/20], Training Loss: 0.2996, Validation Accuracy: 87.10%
Epoch [3/20], Training Loss: 0.1363, Validation Accuracy: 92.96%
Epoch [4/20], Training Loss: 0.0493, Validation Accuracy: 93.84%
Epoch [5/20], Training Loss: 0.0264, Validation Accuracy: 93.55%
Epoch [6/20], Training Loss: 0.0131, Validation Accuracy: 92.38%
Epoch [7/20], Training Loss: 0.0135, Validation Accuracy: 89.74%
Epoch [8/20], Training Loss: 0.0111, Validation Accuracy: 92.67%
Epoch [9/20], Training Loss: 0.0081, Validation Accuracy: 93.84%
Epoch [10/20], Training Loss: 0.0014, Validation Accuracy: 94.13%
Epoch [11/20], Training Loss: 0.0014, Validation Accuracy: 93.84%
Epoch [12/20], Training Loss: 0.0006, Validation Accuracy: 94.43%
Epoch [13/20], Training Loss: 0.0005, Validation Accuracy: 94.13%
Epoch [14/20], Training Loss: 0.0005, Validation Accuracy: 93.26%
Epoch [15/20], Training Loss: 0.0013, Validation Accuracy: 94.43%
Epoch [16/20], Trai

## 4. submission 파일로 저장

In [11]:
import os
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
import torch

# CPU 또는 GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터셋 경로 설정 => test/0 => test/a 로 수정 => 코드 절차 확인 목적

path_test_0 = "/content/test/0"
test_folder_name = "a"
os.rename(path_test_0, os.path.join(os.path.dirname(path_test_0), test_folder_name))


test_dir = './test/a'

# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG는 224x224 크기의 이미지를 입력으로 사용합니다.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 모델을 CPU 모드로 설정
vgg.to(device)
vgg.eval()

# 예측 결과를 저장할 리스트
predictions = []

# 테스트 데이터셋에 대한 예측 수행
for filename in sorted(os.listdir(test_dir)):
    # 이미지 파일 경로 설정
    img_path = os.path.join(test_dir, filename)

    # 이미지 파일 열기 및 변환 적용
    img = Image.open(img_path)
    img = transform(img)

    # 이미지를 모델에 입력하여 예측 수행
    img = img.unsqueeze(0)  # 배치 차원 추가
    img = img.to(device)     # 입력 데이터를 같은 장치에 올리기
    with torch.no_grad():
        output = vgg(img)
        _, predicted = torch.max(output, 1)
        predictions.append(predicted.item())

# CSV 파일에 저장
submission_df = pd.DataFrame({'index': range(1, len(predictions) + 1), 'label': predictions})
submission_df.to_csv('submission.csv', index=False)